# Lesson 2: Querying Knowledge Graphs with Cypher

### Import packages and set up Neo4

In [4]:
from dotenv import load_dotenv
import os

from langchain_community.graphs import Neo4jGraph

# Warning control
import warnings
warnings.filterwarnings("ignore")

In [5]:
load_dotenv('.env', override=True)
NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
NEO4J_DATABASE = os.getenv('NEO4J_DATABASE')

- Initialize a knowledge graph instance using LangChain's Neo4j integration

In [6]:
kg = Neo4jGraph(
    url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD, database=NEO4J_DATABASE
)

### Querying the movie knowledge graph 
- Match all nodes in the graph

In [21]:
# returns how many nodes in the graph
# MATCH is to match on a pattern to search
# (n) is the smallest entity to MATCH which is a node

cypher = """
  MATCH (n) 
  RETURN count(n)
  """

In [22]:
result = kg.query(cypher)

result

[{'count(n)': 171}]

In [23]:
# count(n) is just the same as COUNT(col) in SQL
# You can even do renaming with AS just like in SQL
cypher = """
  MATCH (n) 
  RETURN count(n) AS numberOfNodes
  """

In [24]:
result = kg.query(cypher)

result

[{'numberOfNodes': 171}]

- Match only the `Movie` nodes by specifying the node label

In [25]:
cypher = """
  MATCH (n:Movie)
  RETURN count(n) AS numberOfMovies
"""

kg.query(cypher)

[{'numberOfMovies': 38}]

- Change the variable name in the node pattern match for improved readability

In [26]:
cypher = """
  MATCH (m:Movie) 
  RETURN count(m) AS numberOfMovies
  """
kg.query(cypher)

[{'numberOfMovies': 38}]

- Match only the `Person` nodes

In [30]:
cypher = """
  MATCH (people:Person)
  RETURN count(people) AS numberOfPeople
"""

kg.query(cypher)

[{'numberOfPeople': 133}]

- Match a single person by specifying the value of the `name` property on the `Person` node

In [32]:
cypher = """
  MATCH (tom:Person {name:"Tom Hanks"})
  RETURN tom
"""

kg.query(cypher)

[{'tom': {'born': 1956, 'name': 'Tom Hanks'}}]

- Match a single `Movie` by specifying the value of the `title` property

In [33]:
cypher = """
  MATCH (cloudAtlas:Movie {title:"Cloud Atlas"}) 
  RETURN cloudAtlas
  """
kg.query(cypher)

[{'cloudAtlas': {'tagline': 'Everything is connected',
   'votes': 109,
   'title': 'Cloud Atlas',
   'released': 2012}}]

- Return only the `released` property of the matched `Movie` node

In [34]:
cypher = """
  MATCH (cloudAtlas:Movie {title:"Cloud Atlas"}) 
  RETURN cloudAtlas.released
  """
kg.query(cypher)

[{'cloudAtlas.released': 2012}]

- Return two properties

In [35]:
cypher = """
  MATCH (cloudAtlas:Movie {title:"Cloud Atlas"}) 
  RETURN cloudAtlas.released, cloudAtlas.tagline
  """
kg.query(cypher)

[{'cloudAtlas.released': 2012,
  'cloudAtlas.tagline': 'Everything is connected'}]

### Cypher patterns with conditional matching

In [36]:
cypher = """
  MATCH (nineties:Movie) 
  WHERE nineties.released >= 1990 
    AND nineties.released < 2000 
  RETURN nineties.title
  """
kg.query(cypher)

[{'nineties.title': 'The Matrix'},
 {'nineties.title': "The Devil's Advocate"},
 {'nineties.title': 'A Few Good Men'},
 {'nineties.title': 'As Good as It Gets'},
 {'nineties.title': 'What Dreams May Come'},
 {'nineties.title': 'Snow Falling on Cedars'},
 {'nineties.title': "You've Got Mail"},
 {'nineties.title': 'Sleepless in Seattle'},
 {'nineties.title': 'Joe Versus the Volcano'},
 {'nineties.title': 'When Harry Met Sally'},
 {'nineties.title': 'That Thing You Do'},
 {'nineties.title': 'The Birdcage'},
 {'nineties.title': 'Unforgiven'},
 {'nineties.title': 'Johnny Mnemonic'},
 {'nineties.title': 'The Green Mile'},
 {'nineties.title': 'Hoffa'},
 {'nineties.title': 'Apollo 13'},
 {'nineties.title': 'Twister'},
 {'nineties.title': 'Bicentennial Man'},
 {'nineties.title': 'A League of Their Own'}]

### Pattern matching with multiple nodes

In [13]:
cypher = """
  MATCH (actor:Person)-[:ACTED_IN]->(movie:Movie)
  RETURN actor.name, movie.title LIMIT 10
"""

kg.query(cypher)

[{'actor.name': 'Emil Eifrem', 'movie.title': 'The Matrix'},
 {'actor.name': 'Hugo Weaving', 'movie.title': 'The Matrix'},
 {'actor.name': 'Laurence Fishburne', 'movie.title': 'The Matrix'},
 {'actor.name': 'Carrie-Anne Moss', 'movie.title': 'The Matrix'},
 {'actor.name': 'Keanu Reeves', 'movie.title': 'The Matrix'},
 {'actor.name': 'Hugo Weaving', 'movie.title': 'The Matrix Reloaded'},
 {'actor.name': 'Laurence Fishburne', 'movie.title': 'The Matrix Reloaded'},
 {'actor.name': 'Carrie-Anne Moss', 'movie.title': 'The Matrix Reloaded'},
 {'actor.name': 'Keanu Reeves', 'movie.title': 'The Matrix Reloaded'},
 {'actor.name': 'Hugo Weaving', 'movie.title': 'The Matrix Revolutions'}]

Note that the first person Emil Eifrem is not an actor in the movie The Matrix, instead he is the founder of Neo4J

In [17]:
cypher = """
  MATCH (tom:Person {name:"Tom Hanks"})-[:ACTED_IN]->(tomHanksMovies:Movie)
  RETURN tom.name, tomHanksMovies.title
"""
kg.query(cypher)

[{'tom.name': 'Tom Hanks', 'tomHanksMovies.title': 'Apollo 13'},
 {'tom.name': 'Tom Hanks', 'tomHanksMovies.title': "You've Got Mail"},
 {'tom.name': 'Tom Hanks', 'tomHanksMovies.title': 'A League of Their Own'},
 {'tom.name': 'Tom Hanks', 'tomHanksMovies.title': 'Joe Versus the Volcano'},
 {'tom.name': 'Tom Hanks', 'tomHanksMovies.title': 'That Thing You Do'},
 {'tom.name': 'Tom Hanks', 'tomHanksMovies.title': 'The Da Vinci Code'},
 {'tom.name': 'Tom Hanks', 'tomHanksMovies.title': 'Cloud Atlas'},
 {'tom.name': 'Tom Hanks', 'tomHanksMovies.title': 'Cast Away'},
 {'tom.name': 'Tom Hanks', 'tomHanksMovies.title': 'The Green Mile'},
 {'tom.name': 'Tom Hanks', 'tomHanksMovies.title': 'Sleepless in Seattle'},
 {'tom.name': 'Tom Hanks', 'tomHanksMovies.title': 'The Polar Express'},
 {'tom.name': 'Tom Hanks', 'tomHanksMovies.title': "Charlie Wilson's War"}]

In [20]:
# querying people who acted in moviews which Tom Hanks has also acted in
cypher = """
  MATCH (tom:Person {name: "Tom Hanks"})-[:ACTED_IN]->(m)<-[:ACTED_IN]-(coActor)
  RETURN tom.name, coActor.name
"""
kg.query(cypher)

[{'tom.name': 'Tom Hanks', 'coActor.name': 'Ed Harris'},
 {'tom.name': 'Tom Hanks', 'coActor.name': 'Gary Sinise'},
 {'tom.name': 'Tom Hanks', 'coActor.name': 'Kevin Bacon'},
 {'tom.name': 'Tom Hanks', 'coActor.name': 'Bill Paxton'},
 {'tom.name': 'Tom Hanks', 'coActor.name': 'Parker Posey'},
 {'tom.name': 'Tom Hanks', 'coActor.name': 'Greg Kinnear'},
 {'tom.name': 'Tom Hanks', 'coActor.name': 'Meg Ryan'},
 {'tom.name': 'Tom Hanks', 'coActor.name': 'Steve Zahn'},
 {'tom.name': 'Tom Hanks', 'coActor.name': 'Dave Chappelle'},
 {'tom.name': 'Tom Hanks', 'coActor.name': 'Madonna'},
 {'tom.name': 'Tom Hanks', 'coActor.name': "Rosie O'Donnell"},
 {'tom.name': 'Tom Hanks', 'coActor.name': 'Geena Davis'},
 {'tom.name': 'Tom Hanks', 'coActor.name': 'Bill Paxton'},
 {'tom.name': 'Tom Hanks', 'coActor.name': 'Lori Petty'},
 {'tom.name': 'Tom Hanks', 'coActor.name': 'Nathan Lane'},
 {'tom.name': 'Tom Hanks', 'coActor.name': 'Meg Ryan'},
 {'tom.name': 'Tom Hanks', 'coActor.name': 'Liv Tyler'},
 {'t